In [ ]:
import os, asyncio, json
from pydantic import BaseModel, Field

from grasp_agents.openai.responses.openai_responses_api import (
    OpenAIResponsesLLM,
    OpenAIResponsesLLMSettings,
    OpenAIReasoning,
)
from grasp_agents.openai.responses.responses_content_converters import (
    to_responses_content,
)

from grasp_agents.openai import OpenAIReasoning
from grasp_agents.typing.message import UserMessage, Role
from openai.types.responses.response_stream_event import ResponseStreamEvent
from openai.types.responses.response_text_done_event import ResponseTextDoneEvent
from openai.types.responses.response_reasoning_summary_text_delta_event import (
    ResponseReasoningSummaryTextDeltaEvent,
)
from openai.types.responses.response_reasoning_summary_text_done_event import (
    ResponseReasoningSummaryTextDoneEvent,
)
from openai.types.responses.response_function_call_arguments_done_event import (
    ResponseFunctionCallArgumentsDoneEvent,
)
from openai.types.responses.response_output_item_done_event import (
    ResponseOutputItemDoneEvent,
)

from grasp_agents.openai.message_converters import to_api_user_message

In [ ]:
assert os.getenv("OPENAI_API_KEY"), "Please set OPENAI_API_KEY"


# Dummy function tool schema
class WeatherInput(BaseModel):
    model_config = {"extra": "forbid"}
    location: str = Field(..., description="City and country (e.g., Paris, France)")


function_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for a given location.",
    "parameters": WeatherInput.model_json_schema(),
    "strict": True,
}

# Build Responses input message (user)
user_content = to_responses_content("tell me a joke")
input_messages = [
    {
        "type": "message",
        "role": "user",
        "content": user_content,
    }
]

reasoning = {}

# Initialize the Responses LLM
llm = OpenAIResponsesLLM(
    model_name="openai/gpt-5",
    llm_settings=OpenAIResponsesLLMSettings(
        reasoning=OpenAIReasoning(effort="high", summary="detailed")
    ),
    apply_response_schema_via_provider=False,
)


async def run_stream():
    print("Streaming raw Responses events...")
    async for evt in await llm._get_api_completion_stream(
        api_messages=input_messages,
        response_id=None,
        api_tools=[function_tool],
        api_tool_choice="auto",
        api_response_schema=None,
        reasoning=OpenAIReasoning(effort="high", summary="detailed"),
    ):
        t = evt.type
        print(evt)
        line = t
        if isinstance(evt, ResponseTextDoneEvent):
            line += f" text={evt.text!r}"
        if isinstance(evt, ResponseReasoningSummaryTextDeltaEvent):
            line += f" reasoning_delta={evt.delta!r}"
        if isinstance(evt, ResponseReasoningSummaryTextDoneEvent):
            line += f" reasoning_text={evt.text!r}"
        if isinstance(evt, ResponseFunctionCallArgumentsDoneEvent):
            line += f" args={evt.arguments!r}"
        if isinstance(evt, ResponseOutputItemDoneEvent):
            item = evt.item
            try:
                if item.type == "function_call":
                    line += f" function_call name={item.name} args={item.arguments}"
            except Exception:
                pass
        # print(line)


# Run
await run_stream()


In [ ]:
# --- Repro: continuation stream that hangs without inline function_call_output ---
from pydantic import BaseModel, Field
import asyncio, json
from openai import AsyncOpenAI
from openai.types.responses.response_stream_event import ResponseStreamEvent
from openai.types.responses.response_output_item_done_event import (
    ResponseOutputItemDoneEvent,
)

client = AsyncOpenAI()  # uses OPENAI_API_KEY


# Tool schemas
class AskStudentInput(BaseModel):
    model_config = {"extra": "forbid"}
    question: str = Field(..., description="Question to ask")


ask_student_tool = {
    "type": "function",
    "name": "ask_student",
    "description": "Ask a question to the student.",
    "parameters": AskStudentInput.model_json_schema(),
    "strict": True,
}

# Minimal final answer tool schema (string payload)
final_answer_tool = {
    "type": "function",
    "name": "final_answer",
    "description": "Provide the final answer.",
    "parameters": {
        "type": "object",
        "properties": {"answer": {"type": "string"}},
        "required": ["answer"],
        "additionalProperties": False,
    },
    "strict": True,
}

tools = [ask_student_tool, final_answer_tool]


In [ ]:
# 1) First turn (non-stream): get a function_call
seed_input = [
    {
        "type": "message",
        "role": "user",
        "content": [{"type": "input_text", "text": "Start by asking me a question."}],
    }
]
resp1 = await client.responses.create(
    model="openai/gpt-5",
    input=seed_input,
    tools=tools,
    tool_choice="auto",
)

# Extract function_call
fc1 = next(
    (o for o in (resp1.output or []) if getattr(o, "type", "") == "function_call"), None
)
assert fc1 is not None, "Expected a function_call in first response"
resp_id1 = resp1.id
call_id1 = fc1.call_id or fc1.id
print("Seed response_id:", resp_id1, "call_id:", call_id1, "name:", fc1.name)

# 2) Continue the thread with the tool output (non-stream)
tool_output_1 = {
    "type": "function_call_output",
    "call_id": call_id1,
    "output": json.dumps("jogging"),
}
resp2 = await client.responses.create(
    model="openai/gpt-5",
    previous_response_id=resp_id1,
    input=[tool_output_1],
    tool_choice="auto",
)
resp_id2 = resp2.id
print("Continuation response_id:", resp_id2)


In [ ]:
# 3) Continuation stream that forces the final_answer tool call and hangs without inline output
force_final = {"type": "function", "name": "final_answer"}  # Named tool choice

prompt2 = [
    {
        "type": "message",
        "role": "user",
        "content": [{"type": "input_text", "text": "Now provide the final answer."}],
    }
]

stream_mgr = client.responses.stream(
    model="openai/gpt-5",
    previous_response_id=resp_id2,
    input=prompt2,
    tools=tools,
    tool_choice=force_final,
    timeout=10,
)

async with stream_mgr as stream:
    saw_final_call = False
    try:
        while True:
            evt: ResponseStreamEvent = await asyncio.wait_for(
                stream.__anext__(), timeout=6.0
            )
            print("event:", evt.type)
            if isinstance(evt, ResponseOutputItemDoneEvent):
                item = evt.item
                if getattr(item, "type", "") == "function_call":
                    print(
                        f"function_call name={item.name} call_id={item.call_id or item.id}"
                    )
                    if item.name == "final_answer":
                        saw_final_call = True
                        # Intentionally NOT appending function_call_output inline.
    except asyncio.TimeoutError:
        if saw_final_call:
            print(
                "Timed out waiting for response.completed (server is waiting for inline function_call_output)."
            )
        else:
            print("Timed out before final tool call; re-run previous cell if needed.")
